In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 44.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 61.8 MB/s eta 0:00:00:00:01
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=89dfd928f5e56e3abb261f63525ffa3e2ebb96b09832cac046645782976617c7
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators
Note: you may need to restart the kernel to use updated packages.


In [3]:
import streamlit as st

In [4]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.impute import SimpleImputer
import plotly.graph_objs as go
import plotly.offline as py
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from imblearn.under_sampling import RandomUnderSampler

In [9]:
df1=pd.read_csv("../input/sepsis50/Sepsis_50p.csv")

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1541827 entries, 0 to 1541992
Data columns (total 22 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Hour         1541827 non-null  int64  
 1   Patient_ID   1541827 non-null  int64  
 2   HR           1541827 non-null  float64
 3   O2Sat        1541827 non-null  float64
 4   Temp         1541827 non-null  float64
 5   MAP          1541827 non-null  float64
 6   Resp         1541827 non-null  float64
 7   BaseExcess   556443 non-null   float64
 8   FiO2         764772 non-null   float64
 9   pH           808270 non-null   float64
 10  PaCO2        786805 non-null   float64
 11  BUN          1492488 non-null  float64
 12  Calcium      1393040 non-null  float64
 13  Creatinine   1491858 non-null  float64
 14  Glucose      1500505 non-null  float64
 15  Magnesium    1407257 non-null  float64
 16  Potassium    1494932 non-null  float64
 17  Hct          1483986 non-null  float64
 18  WB

In [14]:
count_vitals=df1.groupby('Patient_ID',as_index=None)['HR','Temp','Resp','O2Sat','MAP'].count()
null_vitals_ID=count_vitals['Patient_ID'].loc[(count_vitals['HR']==0)|(count_vitals['Temp']==0)|(count_vitals['O2Sat']==0)|(count_vitals['MAP']==0)|(count_vitals['Resp']==0)]
null_vitals_ID=null_vitals_ID.to_list()
for i in null_vitals_ID:
        df1.drop(df1[df1['Patient_ID']==i].index,inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [15]:
X=df1.drop(['Patient_ID','SepsisLabel'], axis=1)
y=df1['SepsisLabel']

In [17]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline

In [19]:
from imblearn import under_sampling, over_sampling
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X,y)
print(sorted(Counter(y_resampled).items()),y_resampled.shape)

[(0, 27444), (1, 27444)] (54888,)


In [20]:
df2 = pd.DataFrame(X_resampled)
imputer = KNNImputer()
imp_data = imputer.fit_transform(df2)
column_names = list(df2)
final_data = pd.DataFrame(imp_data, columns = column_names)

In [21]:
X1= final_data
y = y_resampled
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y,test_size = 0.30,random_state = 100,stratify = y)
classifier = RandomForestClassifier(n_estimators = 40,random_state = 100)
rfc=classifier.fit(X1_train, y1_train)
# Predicting the Test set results
y_pred = classifier.predict(X1_test)
from sklearn.metrics import classification_report
print(classification_report(y1_test, y_pred))
from sklearn.metrics import f1_score
f1_score(y_pred,y1_test)

              precision    recall  f1-score   support

           0       0.91      0.88      0.89      8234
           1       0.88      0.91      0.90      8233

    accuracy                           0.90     16467
   macro avg       0.90      0.90      0.90     16467
weighted avg       0.90      0.90      0.90     16467



0.8970369045489774

In [23]:
import pickle as pkl
pkl.dump(rfc,open("final_model.p","wb"))

In [24]:
pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 11.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=7ccaaa3b27635850bc4bb54f9bc42e926ca87bd394ee2078aa64207401ba7f06
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Note: you may need to restart the kernel to use updated packages.


In [25]:
from pyngrok import ngrok

In [27]:
df1.columns

Index(['Hour', 'Patient_ID', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp',
       'BaseExcess', 'FiO2', 'pH', 'PaCO2', 'BUN', 'Calcium', 'Creatinine',
       'Glucose', 'Magnesium', 'Potassium', 'Hct', 'WBC', 'Platelets',
       'ICU_Type', 'SepsisLabel'],
      dtype='object')

In [29]:
%%writefile healthy-heart-app.py
import streamlit as st
import base64
import sklearn
import numpy as np
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
scal=MinMaxScaler()
#Load the saved model
model=pkl.load(open("final_model.p","rb"))
%%writefile healthy-heart-app.py
import streamlit as st
import base64
import sklearn
import numpy as np
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
scal=MinMaxScaler()
#Load the saved model
model=pkl.load(open("final_model.p","rb"))

st.set_page_config(page_title="Healthy Heart App",page_icon="⚕️",layout="centered",initial_sidebar_state="expanded")



def preprocess(Hour, HR, O2Sat, Temp, MAP, Resp,BaseExcess,FiO2,pH, PaCO2, BUN, Calcium, Creatinine,
 Glucose,Magnesium,Potassium,Hct,WBC,Platelets,ICU_Type): 


    user_input=[Hour HR, O2Sat, Temp, MAP, Resp,BaseExcess,FiO2,pH, PaCO2, BUN, Calcium, Creatinine,
 Glucose,Magnesium,Potassium,Hct,WBC,Platelets,ICU_Type]
    user_input=np.array(user_input)
    user_input=user_input.reshape(1,-1)
    user_input=scal.fit_transform(user_input)
    prediction = model.predict(user_input)

    return prediction

    

       
    # front end elements of the web page 
html_temp = """ 
    <div style ="background-color:pink;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Healthy Heart App</h1> 
    </div> 
    """
      
# display the front end aspect
st.markdown(html_temp, unsafe_allow_html = True) 
st.subheader('by Amlan Mohanty ')
      

    
# following lines create boxes in which user can enter data required to make prediction
hour=st.number_input('Hour')
HR=st.number_input('HR')
O2Sat=st.number_input('O2Sat')
Temp=st.number_input('Temp')
MAP=st.number_input('MAP')
Resp=st.number_input('Resp')
BaseExcess=st.number_input('BaseExcess')
FiO2=st.number_input('FiO2')
pH=st.number_input('pH')
PaCO2=st.number_input('PaCO2')
BUN=st.number_input('BUN')
Calcium=st.number_input('Calcium')
Creatinine=st.number_input('Creatinine')
Glucose=st.number_input('Glucose')
Magnesium=st.number_input('Magnesium')
Potassium=st.number_input('Potassium')
Hct=st.number_input('Hct')
WBC=st.number_input('WBC')
Platelets=st.number_input('Platelets')
ICU_Type=st.number_input('ICU_Type')



#user_input=preprocess(sex,cp,exang, fbs, slope, thal )
pred=preprocess(Hour HR, O2Sat, Temp, MAP, Resp,BaseExcess,FiO2,pH, PaCO2, BUN, Calcium, Creatinine,
 Glucose,Magnesium,Potassium,Hct,WBC,Platelets,ICU_Type)




if st.button("Predict"):    
  if pred[0] == 0:
    st.error('Warning! You have high risk of getting a heart attack!')
    
  else:
    st.success('You have lower risk of getting a heart disease!')
    
   



st.sidebar.subheader("About App")

st.sidebar.info("This web app is helps you to find out whether you are at a risk of developing a heart disease.")
st.sidebar.info("Enter the required fields and click on the 'Predict' button to check whether you have a healthy heart")
st.sidebar.info("Don't forget to rate this app")



feedback = st.sidebar.slider('How much would you rate this app?',min_value=0,max_value=5,step=1)

if feedback:
  st.header("Thank you for rating the app!")
  st.info("Caution: This is just a prediction and not doctoral advice. Kindly see a doctor if you feel the symptoms persist.") 

        


Overwriting healthy-heart-app.py


In [30]:
!nohup streamlit run healthy-heart-app.py &
url = ngrok.connect(port='8501')
url

OSError: Background processes not supported.